#Réalisé par :


*   Rayhane Talbi


---

#Outils :
* Google collab
* Python3




# **Librairies**

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.4MB 7.0MB/s 
     |████████████████████████████████| 890kB 17.2MB/s 
     |████████████████████████████████| 2.9MB 15.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=97263d2528f8a6a7a61a22df5a38d951b8875c1c1a511511afb903ea08042fb6
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!pip install torch

In [ ]:
import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

In [ ]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

# **Web Scrapping**

Afin de disposer d'un nombre de questions necessaires, nous avons décidé de travailler sur 3 sites différents. L'avantage est que nous obtenons alors un grand nombre de questions diverses.


## **SITE 1**

In [ ]:
# user agent d'un navigateur moderne pour les requetes
header = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.80 Safari/537.36'}

# URL du premier site
URL = 'https://onlinecoursetutorials.com/interview-questions/natural-language-processing-interview-questions-and-answers/'

httpx = requests.get(URL, headers=header)
soup = BeautifulSoup(httpx.text, 'html.parser')

# Récupère les elements de la classe the_content
questionsSoup = soup.find(class_='the_content')

# L'ensemble des questions se trouvent dans tr
questionsSoup_tr = questionsSoup.find_all('tr')
df = pd.DataFrame(columns=['Question', 'Answer'], index = np.arange(200))
cpt_ligne = 0
cpt_df = 0
for question_answer in questionsSoup_tr:
  # Remove \n
  tmp = question_answer.get_text().replace("\n", "")
  # This question has no answer
  if( "What are punctuation’s ? How can you remove it ?" not in tmp):
    if(tmp != ""):
      # Alternance de question et réponse
      if(cpt_ligne%2==0 and ("What" in tmp or "?" in tmp)):
        df['Question'].loc[cpt_df] = tmp
        cpt_ligne = 0
      else:
        if(cpt_ligne%2 ==0):
          tmp2 = df['Answer'].loc[cpt_df-1]
          df['Answer'].loc[cpt_df-1] = tmp + "-" + tmp2
          cpt_ligne = 1
        else:
          df['Answer'].loc[cpt_df] = tmp 
          cpt_df = cpt_df + 1
          cpt_ligne = 1
      cpt_ligne = cpt_ligne + 1

## **SITE 2**


In [ ]:
# URL du second site
URL = 'https://www.wisdomjobs.com/e-university/natural-language-processing-interview-questions.html'

httpx = requests.get(URL, headers=header)
soup = BeautifulSoup(httpx.text, 'html.parser')
#print(BeautifulSoup(httpx.content, 'html'))
questionsSoup = soup.find(class_='interview_questions')
questionsSoup = questionsSoup.get_text().split('\n')
questionsSoup = [e for e in questionsSoup if(e.strip()!='' and "adsbygoogle" not in e and "Python" not in e and 'Tutorial' not in e and 'Interview' not in e)]

for q in questionsSoup:
  if 'Question' in q and 'Answer' not in q:
    cpt_df += 1
    df['Question'].loc[cpt_df] = q
  else:
    if(str( df['Answer'].loc[cpt_df])!= "nan"):
      tmp2 = df['Answer'].loc[cpt_df]
      df['Answer'].loc[cpt_df] = q + "-" + tmp2
    else:
      df['Answer'].loc[cpt_df] = q 

## **Site 3**


Le webscraping pour ce site est bien plus compliqué que les deux derniers, mais il nous a fournit un nombre de questions et une diversité non négligeable.

In [ ]:
# URL du troisième site
URL3 = 'https://www.mygreatlearning.com/blog/nlp-interview-questions/'

httpx3 = requests.get(URL3, headers=header)
soup3 = BeautifulSoup(httpx3.text, 'html.parser')
#print(BeautifulSoup(httpx.content, 'html'))
questionsSoup3 = soup3.find(class_='td-post-content')

paragraphQSoup = questionsSoup3.find_all('p') # Paragraphe avec les questions et reponsent
listParag = list()
for i in paragraphQSoup:
  tmp = i.get_text().strip() # Convertion sous forme de text et efface les espaces
  listParag.append(re.split(r'r\d{1}', tmp)) # Split les elements par format de la question (1. 2. ...)

# Les 2 premières lignes ne représentent pas des QA et les elements suivant doivent être préparés pour être modifiés
td = listParag[2:28] # Première partie
# QA contient des questions et réponses au format correct Question puis réponse alternée
QA = listParag[28:]


In [ ]:
# Seconde partie
# Ici on ne va travailler que sur les elements de la liste de questions au mauvais format
# Convertit la liste de liste en liste
flat_list = [item for sublist in td for item in sublist]
# Convertit la liste en chaine de caractere 
s = ""
for i in flat_list:
  s = s + i

l = list(range(1, 39))
l = [str(e) + "." for e in l]
listQA = list()
# On split les elements de la liste par numéro de question
for i in reversed(l):
  listQA.append(s.split(i)[1:])

# On ajoute le numéro de question aux questions correctement split
cpt = 0
QA_better = list()
for i in reversed(listQA):
  if (len(i) ==1):
    QA_better.append(l[cpt] + i[0])
  cpt =  cpt + 1

# On efface pour chaque question les questions suivantes
cpt = 10
for i in range(len(QA_better)-2):
  tmp = QA_better[i+1].replace(l[cpt+1], "")
  QA_better[i] =  QA_better[i].replace(QA_better[i+1], "")

# On split chaque quest
list_QA_split = list()
for i in QA_better:
  list_QA_split.append(i.split("Ans"))
  if(len(i.split("Ans "))> 2):
    print("ALAID")
list_QA_split
flat_list = [item for sublist in list_QA_split for item in sublist]
# Contient les questions 10 à 38, alterne question et reponse

tmp  = flat_list + [item for sublist in QA for item in sublist]
ls = list(range(10,50))
indice_question = 0
for i in range(len(tmp)-1):
  if(indice_question%2 == 0):
    # C'est une question et doit donc contenir un nombre
    if(str(ls[indice_question]) not in tmp[i]):
      # Alors c'est une réponse 
      print(ls[indice_question])
      print(tmp[i])
      tmp[i-1] = tmp[i-1] + " " +  tmp[i]
      tmp[i] =""
    else:
      indice_question = indice_question + 1
  else:
    # C'est une réponse
    # Recupère la valeur de la réponse dans la question
    if( "a)" in tmp[i]):
      first = tmp[i-1].split('a.')
      second = first[-1].split('b.')
      tmp[i] = tmp[i].replace("a)", "a)" + second[0]+ " ")    
    elif( 'b)' in tmp[i]):
      first = tmp[i-1].split('b.')
      second = first[-1].split('c.')
      tmp[i] = tmp[i].replace("b)", "b)" + second[0]+ " ")
    elif( 'c)' in tmp[i]):
      first = tmp[i-1].split('c.')
      second = first[-1].split('d.')
      tmp[i] = tmp[i].replace("c)", "c)" + second[0]+ " ")   
    elif( 'd)' in tmp[i]):
      first = tmp[i-1].split('d.')
      second = first[-1]
      tmp[i] = tmp[i].replace("d)", "d)" + second+ " ")    

In [ ]:
# Insertion des questions réponses dans le dataframe
# Seconde partie des questions
cpt_df += 1
for i in tmp:
  try:
    int(i[0])
    cpt_df += 1
    df['Question'].loc[cpt_df] = i
  except:
    df['Answer'].loc[cpt_df] = i

# Première partie
cpt_df += 1
reponse=""
for l in range(len(QA)):
  question=""
  try: 
    int(QA[l][0][0])
  #Si ça ne commence pas par un numéro c'est une réponse
  except:
    reponse+=QA[l][0]
    
  else:
    if (l != 0):
      df['Answer'].loc[cpt_df] = reponse
      reponse=""
    cpt_df += 1
    question = QA[l][0].split('.')[1:]
    q = question[0]
    df['Question'].loc[cpt_df] = q

df['Answer'].loc[cpt_df] = reponse

# Efface les cellules vides - allocation à 200
df = df.dropna()

On obtient un dataframe composé des questions et réponses de 3 différents sites. Il n'est pas nécéssaire de préprocess le texte puisque Bert s'en chargera par la suite.

In [ ]:
df

,Question,Answer
0,What is NLP(natural language processing) ?,Natural language processing is a subfield of c...
1,What is applications of NLP ?,"Text classification, Text summarization, Name ..."
2,What is tokenization ?,Splitting the sentence into words
3,What is stemming ?,Stemming is the process of reducing a word to ...
4,What is lemmatizing ?,Lemmatizing is also same like stemming but the...
...,...,...
114,Which NLP model gives best accuracy?,Naive Bayes Algorithm has the highest accuracy...
115,What are the major tasks of NLP?,"Translation, named entity recognition, relatio..."
116,What are stop words in NLP?,Common words that occur in sentences that add ...
117,What is stemming in NLP?,The process of obtaining the root word from th...


## **Trie**

In [ ]:
# Trie le dataframe dans l'ordre croissant de la taille des questions 
s = df.Answer.str.len().sort_values().index # Renvoie les indexes
ds = df.reindex(s) # Recupère les indexes 
ds  = ds.reset_index() # Reset les indexes
df = ds[:81] # Recupère une questions ayant les réponses les plus courtes pour un fonctionnement optimal de Bert

# **Bert**



In [ ]:
#Model
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

#Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [ ]:
#fonction permettant l'application de bert sur une question et une chaine de caractère contenant les réponses 
def bert(question,reponse):
  encoding = tokenizer.encode_plus(text=question,text_pair=reponse, add_special=True)

  inputs = encoding['input_ids']  #Token embeddings
  sentence_embedding = encoding['token_type_ids']  #Segment embeddings
  tokens = tokenizer.convert_ids_to_tokens(inputs) #input tokens

  x=model(input_ids=torch.tensor([inputs]), token_type_ids=torch.tensor([sentence_embedding]))
  start_index = torch.argmax(x[0])

  end_index = torch.argmax(x[1])

  answer = ' '.join(tokens[start_index:end_index+1])
  
  corrected_answer = ''

  for word in answer.split():
      
      #If it's a subword token
      if word[0:2] == '##':
          corrected_answer += word[2:]
      else:
          corrected_answer += ' ' + word
  return corrected_answer

Pour utiliser Bert, on doit donner a notre modèle une question et une chaine de caractère contenant les réponses aux potentielles questions.

Cepandant, Bert limite la quantité max de tokens de la liste de réponses à 512, nous empêchant de l'utiliser directement sur toutes nos réponses. On doit réaliser l'algorithme de Bert sur plusieurs subsets de réponses, plutôt que sur l'ensemble. 

On considérera par la suite l'ensemble des réponses retournées par Bert comme nouvel ensemble de réponses possible a notre question. On réalisera cette opération tant que la liste de réponses pottentielles est trop grande pour Bert.

In [ ]:
#fonction permettant de diviser la listes de réponse via Bert en sous listes de tailles sufisament petites pour appliquer Bert dessus
def reduc(reponseBertamodif):
  i=0
  reponses=[]
  answer=''
  while (i<len(reponseBertamodif)):
    if (len(answer)<300 and ( len(reponseBertamodif[i]) + len(answer))<300 ):
      try:
        answer+=reponseBertamodif[i]
      except:
        pass

    else:
      reponses.append(answer)
      answer=''
      i-=1
    i+=1
    
  return reponses

In [ ]:
#Question à poser a notre algorithme :
question = 'What is NLP '

#On extrait les réponses du dataframe, que l'on enregistre sous forme de liste
i=0
reponses=[]
answer=''
while (i<len(df)):
  if (len(answer)<300 and ( len(df['Answer'][i]) + len(answer))<300 ):
    try:
      answer+=df['Answer'][i]
    except:
      pass
  else:
    reponses.append(answer)
    answer=''
    i-=1
  i+=1
  
#On obtient la variable reponse qui contient une liste de réponses de taille suffisament petite pour Bert
#On réalise Bert sur toutes les sous listes de réponses
reponseBert =''
reponseBertTableau=[]
for i in range(len(reponses)):
  result = bert(question,reponses[i])+'.'
  reponseBertTableau.append(result)
  reponseBert+=result


vermines = ['','.',',']
#Tant que les différentes réponses via bert on une taille trop élevé pour utiliser l'algorithme une seule fois,
#On redivise nos réponses en listes de réponses
while len(reponseBert)>300 :
  reponses = reduc(reponseBertTableau)
  tmp=[]
  reponseBert=''
  for i in range(len(reponses)):
    if (reponses[i] not in vermines):
      result=bert(question,reponses[i])+'.'
      reponseBert+=result
      tmp.append(result)
  reponseBertTableau=tmp

finalAnswer = bert(question,reponseBert)
print(question)
print(finalAnswer)


Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not recognized.
Keyword arguments {'add_special': True} not reco

What is NLP 
 natural language processing


In [ ]:
finalAnswer

' natural language processing'